### Group Assignment 3 (20%)


#### Group Members

-   Ezlan Zulfiqree bin Hashim (17192056)
-   Nurulnadiah binti Sukerman (S2178467)
-   Nur Dhania Kamalia Kamarol (S2121964)


#### 1. Instantiate fetus training and testing datasets (Ezlan)


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn import preprocessing


# Define a custom dataset class for the fetus dataset
class FetusDataset(Dataset):
    def __init__(self, csv_file):
        le = preprocessing.LabelEncoder()
        csv_data = pd.read_csv(csv_file)
        csv_features = csv_data.iloc[:, 1:].values
        csv_labels = csv_data.iloc[:, 0].values
        csv_labels = le.fit_transform(csv_labels)
        self.features = torch.tensor(csv_features)
        self.labels = torch.tensor(csv_labels)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


# Define the paths to the training and testing CSV files
train_csv_file = "Fetus_trainingdata.csv"
test_csv_file = "Fetus_testingdata.csv"

# Create training and testing datasets
train_dataset = FetusDataset(train_csv_file)
test_dataset = FetusDataset(test_csv_file)

# Specify the batch size and create data loaders
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# You can now use train_loader and test_loader to iterate through your datasets in batches.

#### 2. Instantiate MLP class with init and forward methods (Ezlan)


In [ ]:
import torch.nn as nn


class MLP(nn.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x


# Define the input size, hidden layer sizes, and output size
input_size = 16  # The number of features in your dataset
hidden1_size = 10  # Number of units in the first hidden layer
hidden2_size = 5  # Number of units in the second hidden layer
output_size = (
    2  # Output size for softmax (assuming binary classification, change if needed)
)


# Instantiate the MLP model
model = MLP(input_size, hidden1_size, hidden2_size, output_size)

# You can use this model for training and prediction tasks with your dataset.

#### 3. Train MLP model with fetus training data (Nadiah)


In [ ]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.SGD(
    model.parameters(), lr=0.01
)  # Stochastic Gradient Descent optimizer

# Training loop
num_epochs = 10  # You can adjust the number of epochs as needed

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch [{epoch + 1}/{num_epochs}] Loss: {total_loss / len(train_loader)}")

print("Training completed.")

#### 4. Evaluate MLP model performance with fetus testing data (Nadiah)


In [ ]:
# Evaluate the model on the testing data
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on the testing data: {accuracy:.2f}%")

#### 5. Tabulate testing accuracy with different hyperparameters (Dhania)


#### 6. Plot training and testing loss, training and testing accuracy across epochs (Dhania)
